

**Ứng dụng SVM cho bài toán phân loại ảnh chó - mèo**



In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:

#Giải nén tập dữ liệu
from zipfile import ZipFile

dataset_train = "/kaggle/input/dogs-vs-cats/train.zip"
    
with ZipFile(dataset_train, 'r') as zip:
    zip.extractall()

In [ ]:
import os
import numpy as np
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
from tqdm import tqdm
import joblib
from sklearn.model_selection import GridSearchCV
import cv2
import seaborn as sns
import time
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split

In [ ]:

folder_path = f"Dataset/"
os.makedirs(folder_path, exist_ok=True)

# define path
confusion_image_path = os.path.join(folder_path, 'confusion matrix.png')
classification_file_path = os.path.join(folder_path, 'classification_report.txt')
model_file_path = os.path.join(folder_path, "svm_model.pkl")

# Path dataset
# dataset_dir = "Dataset/"
# train_dir = os.path.join(dataset_dir, "train")
# test_dir = os.path.join(dataset_dir, "test1")
train_dir = "train"
test_dir = "test1"

In [ ]:

#tải dữ liệu, tiền xử lí, gán nhãn
# dog = 1, cat = 0
train_images = os.listdir(train_dir)
features = []
labels = []
image_size = (50, 50)

# xử lí dữ liệu ảnh huấn luyện
for image in tqdm(train_images, desc="Processing Train Images"):
    if image[0:3] == 'cat' :
        label = 0
    else :
        label = 1
    image_read = cv2.imread(train_dir+"/"+image)
    image_resized = cv2.resize(image_read, image_size)
    image_normalized = image_resized / 255.0
    image_flatten = image_normalized.flatten()
    features.append(image_flatten)
    labels.append(label)

In [ ]:
del train_images

In [ ]:
features = np.asarray(features)
labels = np.asarray(labels)

# train test split
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, shuffle=True, random_state=42)

In [ ]:
del features
del labels

In [ ]:
# PCA, SVM, & Pipeline
n_components = 0.8
pca = PCA(n_components=n_components)
svm = SVC()
pca = PCA(n_components=n_components, random_state=42)
pipeline = Pipeline([
    ('pca', pca),
    ('svm', svm)
])

In [ ]:
param_grid = {
    'pca__n_components': [2, 1, 0.9, 0.8],
    'svm__kernel': ['linear', 'rbf', 'poly', 'sigmoid'],
}

In [ ]:
# training
start_time = time.time()

grid_search = GridSearchCV(pipeline, param_grid, cv=3, verbose=4)
grid_search.fit(X_train, y_train)


end_time = time.time()

In [ ]:
del X_train
del y_train

In [ ]:
# Tính tổng thời gian huấn luyện
training_time = end_time - start_time

# Chuyển đổi thời gian sang giờ, phút, giây
hours, remainder = divmod(training_time, 3600)
minutes, seconds = divmod(remainder, 60)

# In ra thời gian huấn luyện
print(f"Thời gian huấn luyện: {int(hours)} giờ {int(minutes)} phút {seconds:.2f} giây")



In [ ]:
# Lấy các tham số & mô hình tốt nhất
best_pipeline = grid_search.best_estimator_
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print("Best Parameters: ", best_params)
print("Best Score: ", best_score)

In [ ]:

# Lưu mô hình tốt nhất
model_filename = 'best_model.joblib'
joblib.dump(best_pipeline, model_filename)

print(f"Model saved as {model_filename}")

In [ ]:
# Đánh giá với tập dữ liệu test
accuracy = best_pipeline.score(X_test, y_test)
print("Accuracy:", accuracy)

In [ ]:
y_pred = best_pipeline.predict(X_test)

# classification report
target_names = ['Cat', 'Dog']
classification_rep = classification_report(y_test, y_pred, target_names=target_names)
print("Classification Report:\n", classification_rep)

with open(classification_file_path, 'w') as file:
    file.write(classification_rep)

In [ ]:
# Confusion matrix
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
plt.xlabel('Predicted labels')
plt.ylabel('True labels')
plt.savefig(confusion_image_path)
plt.show()